In [18]:
import pandas as pd
import networkx as nx
from collections import defaultdict
import heapq
import functions 

In [31]:
from importlib import reload
reload(functions)

<module 'functions' from 'c:\\Users\\matti\\Documents\\ADM\\HW5\\functions.py'>

In [3]:
df = pd.read_csv(r"AirportsClean.csv")

In [ ]:
#Non ci è necessario tutto il dataset di dati, creiamo quindi un dataframe che sara composto solo dalle colonne Origin_airport,Destination_airport,Distance e Count.
##Ci rendiamo conto che alle volte per la stessa coppia Origin_airport,Destination_airport sono presenti distanze differenti (probabilmente per errori di rilevazione o a causa di modifiche dei territori nel tempo), 
##per risolvere sostituiamo il valore della distanza con il valore della distanza piu frequente (la moda) per coppia di Origin_airport,Destination_airport.
##Il campo count invece rappresentera il conteggio per ogni coppia.

df_util=functions.build_df_for_network(df)
df_util.to_csv("df_util.csv")
df_util.head()

# Costruzione del grafo orientato con pesi
flight_network = functions.build_graph(df_util)

# Ottengo una lista di tutti gli aereoporti distinti
airports = pd.concat([df['Origin_airport'], df['Destination_airport']]).drop_duplicates()

In [ ]:
functions.analyze_centrality(flight_network, "YIP")

In [9]:
df_util.head()

,Origin_airport,Destination_airport,Distance
0,MHK,AMW,254
1,EUG,RDM,103
4,MFR,RDM,156
9,SEA,RDM,228
14,PDX,RDM,116


In [24]:
degree_df = functions.calculate_degree_centrality(flight_network)
print(degree_df[['Airport', 'Out_degree', 'In_degree','Total_degree', 'Normalized_degree']].sort_values(by="Airport"))

    Airport  Out_degree  In_degree  Total_degree  Normalized_degree
513     1B1           1          1             2           0.001377
197     ABE         121        115           236           0.162534
145     ABI          62        126           188           0.129477
43      ABQ         196        181           377           0.259642
74      ABR          26         19            45           0.030992
..      ...         ...        ...           ...                ...
609     YKN           2          1             3           0.002066
204     YNG          92        105           197           0.135675
123     YUM          68         67           135           0.092975
683     ZXX           0          1             1           0.000689
565     ZZV           4          5             9           0.006198

[727 rows x 5 columns]


In [26]:
## Calcolo della Betweenness Centrality ponderata
betweenness_centrality = functions.calculate_betweenness_centrality(flight_network)
#
## Mostra i risultati
betweenness_df = pd.DataFrame(
    betweenness_centrality.items(),
    columns=['Airport', 'Betweenness_Centrality']
)
print(betweenness_df.sort_values(by='Betweenness_Centrality', ascending=False, inplace=False))

    Airport  Betweenness_Centrality
456     YIP                0.323335
191     WFB                0.307999
190     KTN                0.219055
192     MTN                0.092484
410     ATL                0.079322
..      ...                     ...
0       AWX                0.000000
14      HUA                0.000000
13      MIQ                0.000000
12      MPS                0.000000
11      CHI                0.000000

[708 rows x 2 columns]


In [29]:
## Calcolo della Closeness Centrality
closeness_centrality = functions.calculate_closeness_centrality(flight_network)

## Visualizza i risultati
closeness_df = pd.DataFrame(closeness_centrality.items(), columns=['Airport', 'Closeness_Centrality'])
print(closeness_df.sort_values(by='Closeness_Centrality', ascending=False, inplace=False))

    Airport  Closeness_Centrality
484     FVS              0.007463
128     STL              0.001097
134     CMI              0.001096
118     DEC              0.001094
115     SPI              0.001093
..      ...                   ...
668     SME              0.000000
672     AYS              0.000000
669     STK              0.000000
676     ARB              0.000000
726     PHD              0.000000

[727 rows x 2 columns]


In [22]:
pagerank_df = pd.DataFrame(functions.calculate_pagerank(flight_network).items(), columns=['Airport', 'PageRank'])
## Visualizza i risultati
print(pagerank_df.sort_values(by='PageRank', ascending=False))

    Airport  PageRank
196     YIP  0.011969
99      SHV  0.007199
104     MSP  0.007116
131     MEM  0.006640
100     MCI  0.006623
..      ...       ...
667     PMH  0.000206
719     LXN  0.000206
716     MHE  0.000206
714     DQC  0.000206
680     HCA  0.000206

[727 rows x 2 columns]


In [32]:
functions.analyze_centrality(flight_network, "YIP")

{'Betweenness Centrality': 0.32333452216274433,
 'Closeness Centrality': 0.00108300373460128,
 'Degree Centrality': 665,
 'PageRank': 0.011968947291634198}

Ask LLM (eg. ChatGPT) to suggest alternative centrality measures that might be relevant to this task. How can you check that the results given by the LLM are trustable?


La Katz Centrality è una misura di centralità usata per identificare l'importanza di un nodo in una rete, prendendo in considerazione sia il numero di connessioni dirette (ad esempio, i vicini immediati) sia le connessioni indirette (i vicini dei vicini, e così via).

Caratteristiche principali
- Un nodo è importante non solo se ha molte connessioni, ma anche se è connesso a nodi importanti (simile alla Eigenvector Centrality).
- Include un bias: La Katz Centrality aggiunge un termine di bias (detto beta) per garantire che anche nodi con poche connessioni abbiano un punteggio positivo. Questo è utile per grafi disconnessi o reti sparse.


La **Katz Centrality** di un nodo \( v \) è definita come:

$$
C_k(v) = \alpha \sum_{u \in N(v)} C_k(u) + \beta
$$

Dove:
- \( C_k(v) \): La Katz Centrality del nodo \( v \).
- \( N(v) \): I vicini del nodo \( v \) (i nodi direttamente collegati a \( v \)).
- \( alpha \): Un fattore di attenuazione che controlla l'influenza delle connessioni indirette.
- \( beta \): Un termine di bias per garantire che i nodi senza connessioni abbiano un punteggio positivo.

In [250]:
nx.katz_centrality(G, alpha=0.01, beta=1.0, max_iter=1000, tol=1e-6, weight='weight')

PowerIterationFailedConvergence: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 1000 iterations')